# Extração de dados com Web Scraping

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Verifica erros na requisição

    soup = BeautifulSoup(response.text, "html.parser")

#Extrair o conteúdo do corpo (body)
    body_content = soup.find("body")

#Salvar em um arquivo para análise (opcional)
    with open("ans_body_content.html", "w", encoding="utf-8") as file:
        file.write(str(body_content))

    print("Dados do corpo HTML salvos em 'ans_body_content.html'.")

except requests.exceptions.RequestException as e:
    print(f"Erro ao acessar o site: {e}")

Dados do corpo HTML salvos em 'ans_body_content.html'.


In [30]:
link_pdf1 = None
link_pdf2 = None

for a in soup.find_all('a', href=True):
    href = a['href']
    if not href.endswith('.pdf'):
        continue
    
    if 'Anexo_I' in href and link_pdf1 is None:
        link_pdf1 = href
    
    if 'Anexo_II' in href and link_pdf2 is None:
        link_pdf2 = href

    if link_pdf1 and link_pdf2:
        break

# Baixar o PDF, se encontrado
if link_pdf1:
    nome_arquivo = "Anexo_I.pdf"
    # Caso o link seja relativo, adiciona o domínio
    if not link_pdf1.startswith('http'):
        link_pdf1 = "https://www.gov.br" + link_pdf1

    print(f"🔗 Link encontrado: {link_pdf1}")
    pdf_response = requests.get(link_pdf1)
    with open(nome_arquivo, 'wb') as f:
        f.write(pdf_response.content)
    print(f"✅ PDF baixado com sucesso como: {nome_arquivo}")
else:
    print("❌ Link do Anexo I não encontrado.")



if link_pdf2:
    nome_arquivo2 = "Anexo_II.pdf"
        #Caso o link seja relativo, adiciona o domínio
    if not link_pdf2.startswith('http'):
        link_pdf2 = "https://www.gov.br" + link_pdf2

    print(f"🔗 Link encontrado: {link_pdf2}")
    pdf_response = requests.get(link_pdf2)
    with open(nome_arquivo2, 'wb') as f:
        f.write(pdf_response.content)
    print(f"✅ PDF baixado com sucesso como: {nome_arquivo2}")
else:
    print("❌ Link do Anexo II não encontrado.")

🔗 Link encontrado: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
✅ PDF baixado com sucesso como: Anexo_I.pdf
🔗 Link encontrado: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
✅ PDF baixado com sucesso como: Anexo_II.pdf


In [36]:
import requests
import os
from zipfile import ZipFile

# Compacta os dois arquivos em um ZIP
zip_path = "Anexos.zip"
with ZipFile(zip_path, 'w') as zipf:
    zipf.write('./Anexo_I.pdf', arcname="Anexo_I.pdf")
    zipf.write('./Anexo_II.pdf', arcname="Anexo_II.pdf")

print(f"✅ Arquivo ZIP criado com sucesso: {zip_path}")

✅ Arquivo ZIP criado com sucesso: Anexos.zip


# Transformação de Dados

In [7]:
import pandas as pd
import zipfile
import os
from tabula import read_pdf

# Nome do arquivo PDF e do resultado
PDF_PATH = "Anexo_I.pdf"
CSV_PATH = "procedimentos.csv"
ZIP_PATH = "Teste_Lucas_Jaensen_Daros.zip"

# 2.1 Extração de todas as tabelas do PDF
print("Lendo tabelas do PDF...")
tabelas = read_pdf(PDF_PATH, pages='3-181', multiple_tables=True, lattice=True)

# 2.2 Juntar todas em uma tabela única
print("Concatenando dados...")
df_completo = pd.concat(tabelas, ignore_index=True)

# 2.4 Substituir abreviações OD e AMB (ajuste conforme o nome das colunas)
colunas = df_completo.columns

#df_completo.head()
display(df_completo)

Lendo tabelas do PDF...


JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [9]:
print(len(df_completo.columns))

NameError: name 'df_completo' is not defined

In [50]:


# 2.2 Salvar CSV
print("Salvando CSV...")
df_completo.to_csv(CSV_PATH, index=False)



Salvando CSV...


In [51]:
# 2.3 Compactar o CSV em um .zip
print("Compactando CSV em zip...")
with zipfile.ZipFile(ZIP_PATH, 'w') as zipf:
    zipf.write(CSV_PATH)

print(f"Arquivo gerado: {ZIP_PATH}")

Compactando CSV em zip...
Arquivo gerado: Teste_Lucas_Jaensen_Daros.zip


In [58]:
# Detectar colunas com OD e AMB, assumindo que estão nos nomes
for col in colunas:
    df_completo[col] = df_completo[col].replace('OD', 'Seg. Odontológico')
    df_completo[col] = df_completo[col].replace('AMB', 'Seg. Ambulatorial')

In [8]:
display(df_completo)

NameError: name 'df_completo' is not defined

# Teste Banco

In [20]:
import pandas as pd
from sqlalchemy import create_engine
import os

# 1. Caminho da pasta com os CSVs
pasta_csvs = "c:/Users/Lucas/Desktop/1T2024"

# 2. Conexão com o PostgreSQL
usuario = "postgres"
senha = "postgres"
host = "localhost"
porta = "5432"
banco = "postgres"

# 3. Cria a engine de conexão
conexao = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}"
engine = create_engine(conexao)

# 4. Loop para ler e importar todos os CSVs
for arquivo in os.listdir(pasta_csvs):
    if arquivo.lower().endswith('.csv'):
        caminho_csv = os.path.join(pasta_csvs, arquivo)
        print(f"📂 Lendo arquivo: {arquivo}")

        try:
            # Leitura do CSV com encoding correto
            df = pd.read_csv(caminho_csv, sep=';', encoding='latin1')

            # Renomear colunas
            df.rename(columns={
                'DATA': 'data_registro',
                'REG_ANS': 'reg_ans',
                'CD_CONTA_CONTABIL': 'cd_contabeis',
                'DESCRICAO': 'descricao',
                'VL_SALDO_INICIAL': 'vl_saldo_inicial',
                'VL_SALDO_FINAL': 'vl_saldo_final'
            }, inplace=True)

            # Converter números e corrigir encoding de texto
            df['vl_saldo_inicial'] = df['vl_saldo_inicial'].astype(str).str.replace(',', '.').astype(float)
            df['vl_saldo_final'] = df['vl_saldo_final'].astype(str).str.replace(',', '.').astype(float)
            df['descricao'] = df['descricao'].apply(
                lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x
            )

            # Envia para o banco
            df.to_sql("contabeis", engine, if_exists="append", index=False)
            print(f"✅ Dados de '{arquivo}' inseridos com sucesso.")

        except Exception as e:
            print(f"❌ Erro ao processar '{arquivo}': {e}")

print("\n🏁 Importação finalizada.")

📂 Lendo arquivo: 1T2023.csv
✅ Dados de '1T2023.csv' inseridos com sucesso.
📂 Lendo arquivo: 1T2024.csv
✅ Dados de '1T2024.csv' inseridos com sucesso.
📂 Lendo arquivo: 2t2023.csv
✅ Dados de '2t2023.csv' inseridos com sucesso.
📂 Lendo arquivo: 2T2024.csv
✅ Dados de '2T2024.csv' inseridos com sucesso.
📂 Lendo arquivo: 3T2023.csv
✅ Dados de '3T2023.csv' inseridos com sucesso.
📂 Lendo arquivo: 3T2024.csv
✅ Dados de '3T2024.csv' inseridos com sucesso.
📂 Lendo arquivo: 4T2023.csv
✅ Dados de '4T2023.csv' inseridos com sucesso.
📂 Lendo arquivo: 4T2024.csv
✅ Dados de '4T2024.csv' inseridos com sucesso.

🏁 Importação finalizada.


In [26]:
import pandas as pd
from sqlalchemy import create_engine

# Caminho do CSV
csv_path = "c:/Users/Lucas/Desktop/Relatorio_cadop.csv"  # usar / ou raw string

# Conexão PostgreSQL
usuario = "postgres"
senha = "postgres"
host = "localhost"
porta = "5432"
banco = "postgres"

engine = create_engine(f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}")

# Leitura do CSV
df = pd.read_csv(csv_path, sep=';', encoding='latin1')

# Converter datas
df['Data_Registro_ANS'] = pd.to_datetime(df['Data_Registro_ANS'], errors='coerce', dayfirst=True)

# Renomear colunas para coincidir com o banco
df.columns = [
    'registro_ans', 'cnpj', 'razao_social', 'nome_fantasia', 'modalidade',
    'logradouro', 'numero', 'complemento', 'bairro', 'cidade', 'uf', 'cep',
    'ddd', 'telefone', 'fax', 'endereco_eletronico', 'representante',
    'cargo_representante', 'regiao_de_comercializacao', 'data_registro_ans'
]

# Corrigir codificação de todas as colunas do tipo objeto (texto)
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(
        lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x
    )

# Inserir no banco
df.to_sql("operadoras", engine, if_exists="append", index=False)

print("✅ Dados importados com sucesso para a tabela 'operadoras'.")

C:\Users\Lucas\AppData\Local\Temp\ipykernel_20728\1118479709.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['Data_Registro_ANS'] = pd.to_datetime(df['Data_Registro_ANS'], errors='coerce', dayfirst=True)


✅ Dados importados com sucesso para a tabela 'operadoras'.


In [5]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.2, pytest-8.3.5, pluggy-1.5.0
rootdir: Z:\home\lucasjd\Projetos\IntuitiveCare
collected 4 items

tests\test_teste_banco.py EE                                             [ 50%]
tests\test_transformacao_de_dados.py F                                   [ 75%]
tests\test_web_scraping.py .                                             [100%]

=================================== ERRORS ====================================
_______________ ERROR at setup of test_importar_csvs_contabeis ________________
file Z:\home\lucasjd\Projetos\IntuitiveCare\tests\test_teste_banco.py, line 19
  def test_importar_csvs_contabeis(engine):
E       fixture 'engine' not found
>       available fixtures: cache, capfd, capfdbinary, caplog, capsys, capsysbinary, doctest_namespace, monkeypatch, pytestconfig, record_property, record_testsuite_property, record_xml_attribute, recwarn, tmp_path, tmp_path_factory, 

# API